In [4]:
from pykrx import stock
import numpy as np
import pandas as pd
import talib as ta
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#옵션
pd.set_option('display.float_format', '{:.6f}'.format)

In [15]:
# 일봉컬럼 생성
code='005490'
ma_df = stock.get_market_ohlcv("20200101", "20240105", code)
ma_df = ma_df.drop(['등락률'], axis=1)
ma_df

,시가,고가,저가,종가,거래량
날짜,,,,,
2020-01-02,236000,240000,234500,236000,213578
2020-01-03,237500,242000,236500,237000,273831
2020-01-06,234000,234500,230000,231000,217886
2020-01-07,231500,234500,231000,232000,138750
2020-01-08,229000,229500,226500,228000,340244
...,...,...,...,...,...
2023-12-28,495500,502000,493000,499500,560329
2024-01-02,495000,496000,488000,488000,535256
2024-01-03,483000,487000,472500,472500,561452


In [16]:
ma_df['d_close_ma3'] = ta.SMA(ma_df['종가'], timeperiod=3)
ma_df['d_close_ma5'] = ta.SMA(ma_df['종가'], timeperiod=5)
ma_df['d_close_ma10'] = ta.SMA(ma_df['종가'], timeperiod=10)
ma_df['d_close_ma20'] = ta.SMA(ma_df['종가'], timeperiod=20)
ma_df['d_close_ma60'] = ta.SMA(ma_df['종가'], timeperiod=60)
ma_df['d_close_ma120'] = ta.SMA(ma_df['종가'], timeperiod=120)

ma_df = ma_df.drop(['시가','고가','저가','종가','거래량'], axis=1)
ma_df

,d_close_ma3,d_close_ma5,d_close_ma10,d_close_ma20,d_close_ma60,d_close_ma120
날짜,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,234666.6667,NaN,NaN,NaN,NaN,NaN
2020-01-07,233333.3333,NaN,NaN,NaN,NaN,NaN
2020-01-08,230333.3333,232800.0000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-12-28,488500.0000,486500.0000,485500.0000,475275.0000,471908.3333,511675.0000
2024-01-02,493333.3333,486800.0000,488200.0000,475525.0000,471525.0000,512383.3333
2024-01-03,486666.6667,485200.0000,487450.0000,475450.0000,470883.3333,513037.5000


In [17]:
path_=f'C:\\education\\finalproject-dev\\target\\save_data\\A{code}_20230626_20230728.csv'
data_df= pd.read_csv(path_, index_col='날짜')
data_df.index = pd.to_datetime(data_df.index, format='%Y%m%d')
data_df['시간'] = data_df['시간'].astype(str).str.zfill(4) # 시간에 0채우기
data_df = data_df[data_df['시간'] != '1530']
data_df

,시간,시가,고가,저가,종가,거래량,거래대금
날짜,,,,,,,
2023-06-26,0901,391500,393500,391000,392000,10794,4228050000
2023-06-26,0902,392000,392500,390500,392500,8229,3221940000
2023-06-26,0903,392500,395500,392000,395000,18471,7276740000
2023-06-26,0904,395000,396000,394500,395000,8072,3190790000
2023-06-26,0905,395500,395500,393000,393500,5329,2101540000
...,...,...,...,...,...,...,...
2023-07-28,1516,617000,617000,616000,617000,3857,2377850000
2023-07-28,1517,617000,617000,615000,615000,11496,7079100000
2023-07-28,1518,615000,618000,615000,618000,15168,9347970000


In [18]:

# 누적 거래량 계산
data_df['stack_volume'] = data_df.groupby('날짜')['거래량'].cumsum()
data_df['stack_money'] = data_df.groupby('날짜')['거래대금'].cumsum()
data_df

,시간,시가,고가,저가,종가,거래량,거래대금,stack_volume,stack_money
날짜,,,,,,,,,
2023-06-26,0901,391500,393500,391000,392000,10794,4228050000,10794,4228050000
2023-06-26,0902,392000,392500,390500,392500,8229,3221940000,19023,7449990000
2023-06-26,0903,392500,395500,392000,395000,18471,7276740000,37494,14726730000
2023-06-26,0904,395000,396000,394500,395000,8072,3190790000,45566,17917520000
2023-06-26,0905,395500,395500,393000,393500,5329,2101540000,50895,20019060000
...,...,...,...,...,...,...,...,...,...
2023-07-28,1516,617000,617000,616000,617000,3857,2377850000,3920982,2359416690000
2023-07-28,1517,617000,617000,615000,615000,11496,7079100000,3932478,2366495790000
2023-07-28,1518,615000,618000,615000,618000,15168,9347970000,3947646,2375843760000


In [ ]:
# data_df.loc['2022-01-12']

In [19]:
data2_df = pd.read_csv('C:\\education\\finalproject-dev\\20240105기준회사정보2.csv',header=[0,1,2])
data2_df = data2_df.drop(data2_df.iloc[0].name)
# 인덱스가 날짜 데이터를 담고 있는지 확인하고 변환
# 현재 인덱스를 'DatetimeIndex'로 변환
data2_df.index = pd.to_datetime(data2_df.index)
# 날짜 형식을 'YYYYMMDD'로 변경
data2_df.index = pd.to_datetime(data2_df.index, format='%Y%m%d')
data2_df

,상장주식수 (보통)(주),유동주식수(주),유동주식비율(%),시가총액 (티커-상장예정주식수 미포함)(백만원),상장주식수 (보통)(주),유동주식수(주),유동주식비율(%),시가총액 (티커-상장예정주식수 미포함)(백만원),상장주식수 (보통)(주),유동주식수(주),...,유동주식비율(%),시가총액 (티커-상장예정주식수 미포함)(백만원),상장주식수 (보통)(주),유동주식수(주),유동주식비율(%),시가총액 (티커-상장예정주식수 미포함)(백만원),상장주식수 (보통)(주),유동주식수(주),유동주식비율(%),시가총액 (티커-상장예정주식수 미포함)(백만원)
,A060310,A060310,A060310,A060310,A095570,A095570,A095570,A095570,A006840,A006840,...,A003280,A003280,A037440,A037440,A037440,A037440,A238490,A238490,A238490,A238490
,3S,3S,3S,3S,AJ네트웍스,AJ네트웍스,AJ네트웍스,AJ네트웍스,AK홀딩스,AK홀딩스,...,흥아해운,흥아해운,희림,희림,희림,희림,힘스,힘스,힘스,힘스
2022-01-03,"46,271,513","42,475,169",91.8000,"152,696","46,822,295","14,102,592",30.1200,"249,095","13,247,561","4,349,199",...,11.3400,"655,158","13,922,475","9,031,465",64.8700,"92,445","11,312,236","7,582,697",67.0300,"113,122"
2022-01-04,"46,271,513","42,475,169",91.8000,"153,390","46,822,295","14,102,592",30.1200,"249,095","13,247,561","4,349,199",...,11.3400,"646,743","13,922,475","9,031,465",64.8700,"93,420","11,312,236","7,582,697",67.0300,"114,819"
2022-01-05,"46,271,513","42,475,169",91.8000,"155,472","46,822,295","14,102,592",30.1200,"244,881","13,247,561","4,349,199",...,11.3400,"646,743","13,922,475","9,031,465",64.8700,"89,382","11,312,236","7,582,697",67.0300,"113,009"
2022-01-06,"46,271,513","42,475,169",91.8000,"153,159","46,822,295","14,102,592",30.1200,"238,325","13,247,561","4,349,199",...,11.3400,"622,700","13,922,475","9,031,465",64.8700,"89,382","11,312,236","7,582,697",67.0300,"111,426"
2022-01-07,"46,271,513","42,475,169",91.8000,"159,868","46,822,295","14,102,592",30.1200,"239,730","13,247,561","4,349,199",...,11.3400,"631,115","13,922,475","9,031,465",64.8700,"91,192","11,312,236","7,582,697",67.0300,"113,122"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-01,"48,536,642","45,019,594",92.7500,"119,400","45,252,759","16,174,759",35.7400,"225,359","13,247,561","4,373,012",...,15.0800,"601,062","13,922,475","9,982,310",71.7000,"107,482","11,312,236","7,421,496",65.6100,"78,394"
2024-01-02,"48,536,642","45,019,594",92.7500,"155,075","45,252,759","16,174,759",35.7400,"233,052","13,247,561","4,373,012",...,15.0800,"597,456","13,922,475","9,982,310",71.7000,"107,342","11,312,236","7,421,496",65.6100,"79,299"


In [20]:
# 컬럼 인덱스를 재설정
data2_df.columns = pd.MultiIndex.from_tuples(data2_df.columns)

# A_extracted_number와 일치하는 컬럼 선택
matched_columns = data2_df.xs(f'A{code}', level=1, axis=1)

# 필요한 컬럼들을 추출하여 data_df에 추가
for col in matched_columns:
    # 레벨 0의 컬럼 이름을 사용 (예: '상장주식수 (보통)(주)', '유동주식수(주)', 등)
    col_name = col[0]

    # data2_df에서 해당 컬럼의 데이터를 data_df에 추가
    # 컬럼 이름을 새로운 이름으로 지정 (예: '상장주식수', '유동주식수', 등)
    data_df[col_name] = matched_columns[col].astype(str).str.replace(',', '').astype(float)
#컬럼명 변경
data_df.rename(columns={'상장주식수 (보통)(주)': '상장주식수', '유동주식수(주)': '유동주식수', '유동주식비율(%)': '유동주식비율', '시가총액 (티커-상장예정주식수 미포함)(백만원)': '시가총액'}, inplace=True)

# 결과 확인
data_df

,시간,시가,고가,저가,종가,거래량,거래대금,stack_volume,stack_money,상장주식수,유동주식수,유동주식비율,시가총액
날짜,,,,,,,,,,,,,
2023-06-26,0901,391500,393500,391000,392000,10794,4228050000,10794,4228050000,84571230.0000,61480195.0000,72.7000,33828492.0000
2023-06-26,0902,392000,392500,390500,392500,8229,3221940000,19023,7449990000,84571230.0000,61480195.0000,72.7000,33828492.0000
2023-06-26,0903,392500,395500,392000,395000,18471,7276740000,37494,14726730000,84571230.0000,61480195.0000,72.7000,33828492.0000
2023-06-26,0904,395000,396000,394500,395000,8072,3190790000,45566,17917520000,84571230.0000,61480195.0000,72.7000,33828492.0000
2023-06-26,0905,395500,395500,393000,393500,5329,2101540000,50895,20019060000,84571230.0000,61480195.0000,72.7000,33828492.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-28,1516,617000,617000,616000,617000,3857,2377850000,3920982,2359416690000,84571230.0000,61910933.0000,73.2100,52349591.0000
2023-07-28,1517,617000,617000,615000,615000,11496,7079100000,3932478,2366495790000,84571230.0000,61910933.0000,73.2100,52349591.0000
2023-07-28,1518,615000,618000,615000,618000,15168,9347970000,3947646,2375843760000,84571230.0000,61910933.0000,73.2100,52349591.0000


In [22]:
data_df = data_df.join(ma_df, how='left')
data_df['시가총액'] = data_df['종가'] * data_df['상장주식수']
data_df

,시간,시가,고가,저가,종가,거래량,거래대금,stack_volume,stack_money,상장주식수,유동주식수,유동주식비율,시가총액,d_close_ma3,d_close_ma5,d_close_ma10,d_close_ma20,d_close_ma60,d_close_ma120
날짜,,,,,,,,,,,,,,,,,,,
2023-06-26,0901,391500,393500,391000,392000,10794,4228050000,10794,4228050000,84571230.0000,61480195.0000,72.7000,33151922160000.0000,396000.0000,391300.0000,390900.0000,383975.0000,381191.6667,346591.6667
2023-06-26,0902,392000,392500,390500,392500,8229,3221940000,19023,7449990000,84571230.0000,61480195.0000,72.7000,33194207775000.0000,396000.0000,391300.0000,390900.0000,383975.0000,381191.6667,346591.6667
2023-06-26,0903,392500,395500,392000,395000,18471,7276740000,37494,14726730000,84571230.0000,61480195.0000,72.7000,33405635850000.0000,396000.0000,391300.0000,390900.0000,383975.0000,381191.6667,346591.6667
2023-06-26,0904,395000,396000,394500,395000,8072,3190790000,45566,17917520000,84571230.0000,61480195.0000,72.7000,33405635850000.0000,396000.0000,391300.0000,390900.0000,383975.0000,381191.6667,346591.6667
2023-06-26,0905,395500,395500,393000,393500,5329,2101540000,50895,20019060000,84571230.0000,61480195.0000,72.7000,33278779005000.0000,396000.0000,391300.0000,390900.0000,383975.0000,381191.6667,346591.6667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-28,1516,617000,617000,616000,617000,3857,2377850000,3920982,2359416690000,84571230.0000,61910933.0000,73.2100,52180448910000.0000,614333.3333,628600.0000,565700.0000,487300.0000,415208.3333,381750.0000
2023-07-28,1517,617000,617000,615000,615000,11496,7079100000,3932478,2366495790000,84571230.0000,61910933.0000,73.2100,52011306450000.0000,614333.3333,628600.0000,565700.0000,487300.0000,415208.3333,381750.0000
2023-07-28,1518,615000,618000,615000,618000,15168,9347970000,3947646,2375843760000,84571230.0000,61910933.0000,73.2100,52265020140000.0000,614333.3333,628600.0000,565700.0000,487300.0000,415208.3333,381750.0000


In [23]:
data_df['분봉유동비율'] = data_df['거래량']/data_df['유동주식수']
data_df['전봉대비거래량비율'] = data_df['거래량']/data_df['거래량'].shift(1)

data_df['시가퍼센트'] = round((data_df['시가']-data_df['종가'].shift(1))/data_df['종가'].shift(1),4)*100
data_df['고가퍼센트'] = round((data_df['고가']-data_df['종가'].shift(1))/data_df['종가'].shift(1),4)*100
data_df['저가퍼센트'] = round((data_df['저가']-data_df['종가'].shift(1))/data_df['종가'].shift(1),4)*100
data_df['종가퍼센트'] = round((data_df['종가']-data_df['종가'].shift(1))/data_df['종가'].shift(1),4)*100


#캔들상태 = 3 : 양봉, 2 : 보합, 1 : 음봉
data_df['캔들상태'] = np.where(data_df['종가'] > data_df['시가'], 3,
                          np.where(data_df['종가'] == data_df['시가'], 2, 1))

data_df['윗꼬리대비몸통'] = np.where(data_df['캔들상태'] == 3, 
                                  1.0 - (data_df['고가'] - data_df['종가']) / (data_df['종가'] - data_df['시가']),
                                  np.where(data_df['캔들상태'] == 2, 
                                          0.0, 
                                           (1.0 - (data_df['고가'] - data_df['시가']) / (data_df['시가'] - data_df['종가']))*-1)
                              )

data_df['m_close_ma3'] = ta.SMA(data_df['종가'], timeperiod=3)
data_df['m_close_ma5'] = ta.SMA(data_df['종가'], timeperiod=5)
data_df['m_close_ma10'] = ta.SMA(data_df['종가'], timeperiod=10)
data_df['m_close_ma20'] = ta.SMA(data_df['종가'], timeperiod=20)
data_df['m_close_ma39'] = ta.SMA(data_df['종가'], timeperiod=39)
data_df['m_close_ma60'] = ta.SMA(data_df['종가'], timeperiod=60)
data_df['m_close_ma120'] = ta.SMA(data_df['종가'], timeperiod=120)



data_df['캔들몸통'] = np.where(data_df['캔들상태'] == 3, 
                                  (data_df['종가'] - data_df['시가']),
                                  np.where(data_df['캔들상태'] == 2, 
                                          0.0, 
                                           (data_df['시가'] - data_df['종가']))
                              )

data_df['10분봉몸통합'] = data_df['캔들몸통'].rolling(window=10).sum()
data_df['10분봉몸통합'].fillna(0.0, inplace=True)
data_df['효율적분석'] = np.where(data_df['10분봉몸통합'] != 0, 
                                (data_df['종가'] - data_df['종가'].shift(9)) / data_df['10분봉몸통합'], 
                                0.0)
# === 오실레이터 ===
# 스토케스틱 오실레이터 계산
so = ta.STOCH(
    data_df['고가'],
    data_df['저가'],
    data_df['종가'],
    fastk_period=14,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0
)
data_df['stoch_slowk'] = so[0]  # %K
data_df['stoch_slowd'] = so[1]  # %D

# RSI 계산
data_df['RSI'] = ta.RSI(data_df['종가'], 14)

# === 변동성 지표 ===
# 볼린저 밴드 계산
upperband, middleband, lowerband = ta.BBANDS(
    data_df['종가'],
    timeperiod=20,
    nbdevup=2,
    nbdevdn=2,
    matype=0
)
data_df['upperband'] = upperband
data_df['middleband'] = middleband
data_df['lowerband'] = lowerband

# True Range 계산
data_df['true_range'] = ta.TRANGE(data_df['고가'], data_df['저가'], data_df['종가'])

# === 추세 지표 ===
# MACD 계산
macd, macdsignal, macdhist = ta.MACD(
    data_df['종가'],
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)
data_df['macd'] = macd
data_df['macdsignal'] = macdsignal
data_df['macdhist'] = macdhist

# === 모멘텀 지표 ===
# 모멘텀 계산
data_df['MOM'] = ta.MOM(data_df['종가'], timeperiod=10)

# 패러볼릭 SAR 계산
data_df['SAR'] = ta.SAR(data_df['고가'], data_df['저가'], acceleration=0.02, maximum=0.2)

# 상품 채널 지수 (CCI) 계산
data_df['CCI'] = ta.CCI(data_df['고가'], data_df['저가'], data_df['종가'], timeperiod=14)

# === 거래량 지표 ===
# 거래량 균형 (OBV) 계산
data_df['OBV'] = ta.OBV(data_df['종가'], data_df['거래량'])

# === 패턴 인식 ===
# 해머 패턴 식별
data_df['Hammer'] = ta.CDLHAMMER(data_df['시가'], data_df['고가'], data_df['저가'], data_df['종가'])

# 흡수 패턴 식별
data_df['Engulfing'] = ta.CDLENGULFING(data_df['시가'], data_df['고가'], data_df['저가'], data_df['종가'])

# 도지 패턴 식별
data_df['Doji'] = ta.CDLDOJI(data_df['시가'], data_df['고가'], data_df['저가'], data_df['종가'])

In [28]:
selected_columns = data_df[['시간', '시가', '고가', '저가', '종가', '거래량', '거래대금', 'stack_volume', 'stack_money', '상장주식수', '유동주식수', '유동주식비율', '시가총액','분봉유동비율','전봉대비거래량비율']]
selected_columns

,시간,시가,고가,저가,종가,거래량,거래대금,stack_volume,stack_money,상장주식수,유동주식수,유동주식비율,시가총액,분봉유동비율,전봉대비거래량비율
날짜,,,,,,,,,,,,,,,
2023-06-26,0901,391500,393500,391000,392000,10794,4228050000,10794,4228050000,84571230.000000,61480195.000000,72.700000,33151922160000.000000,0.000176,NaN
2023-06-26,0902,392000,392500,390500,392500,8229,3221940000,19023,7449990000,84571230.000000,61480195.000000,72.700000,33194207775000.000000,0.000134,0.762368
2023-06-26,0903,392500,395500,392000,395000,18471,7276740000,37494,14726730000,84571230.000000,61480195.000000,72.700000,33405635850000.000000,0.000300,2.244623
2023-06-26,0904,395000,396000,394500,395000,8072,3190790000,45566,17917520000,84571230.000000,61480195.000000,72.700000,33405635850000.000000,0.000131,0.437009
2023-06-26,0905,395500,395500,393000,393500,5329,2101540000,50895,20019060000,84571230.000000,61480195.000000,72.700000,33278779005000.000000,0.000087,0.660183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-28,1516,617000,617000,616000,617000,3857,2377850000,3920982,2359416690000,84571230.000000,61910933.000000,73.210000,52180448910000.000000,0.000062,0.528429
2023-07-28,1517,617000,617000,615000,615000,11496,7079100000,3932478,2366495790000,84571230.000000,61910933.000000,73.210000,52011306450000.000000,0.000186,2.980555
2023-07-28,1518,615000,618000,615000,618000,15168,9347970000,3947646,2375843760000,84571230.000000,61910933.000000,73.210000,52265020140000.000000,0.000245,1.319415


In [29]:
selected_columns.to_csv('POSCO홀딩스_분봉유동비율,전봉대비거래량비율.csv')

In [ ]:
data_df.replace([np.nan, np.inf, -np.inf],0,inplace=True)
data_df.isnull().sum()

In [ ]:
infinity_mask = data_df.isin([np.inf])

# 인피니티 값을 가진 행 출력
infinity_rows = data_df[infinity_mask.any(axis=1)]
print("인피니티 값을 가진 행:\n", infinity_rows)

In [ ]:
data_df = data_df.rename(columns={'날짜': 'date', '시간': 'time', '시가': 'open', '고가': 'high', '저가': 'low', '종가': 'close', '거래량': 'volume', '거래대금': 'money', '상장주식수': 'stocks', '유동주식수': 'floatingStocks', '유동주식비율': 'floatpercentage', '시가총액': 'market_cap', '분봉유동비율': 'mlr', '전봉대비거래량비율': 'pvr', '시가퍼센트': 'open_ratio', '고가퍼센트': 'high_ratio', '저가퍼센트': 'low_ratio', '종가퍼센트': 'close_ratio', '캔들상태': 'candle' ,'윗꼬리대비몸통': 'hightail_body_ratio','캔들몸통': 'candle_body','10분봉몸통합': 'm_10_sum_candle_body','효율적분석': 'efficiency_analysis'})

In [ ]:
data_df = data_df.loc['2023-07-01':]
data_df

# 전처리

In [ ]:
copy_df = data_df
copy_df

In [ ]:
path1_=f'C:\\education\\finalproject-dev\\target\\save_data\\{code}_전처리_20230703_20230728_최종.csv'

data_df.to_csv(path1_)